<a href="https://colab.research.google.com/github/ZombieSwan/qlora-mistral-finetune/blob/main/qlora-mistral-alpaca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers peft datasets bitsandbytes accelerate --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.5 MB/s eta 0:00:00


In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import get_peft_model, LoraConfig, TaskType

model_name = "mistralai/Mistral-7B-v0.1"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16"
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

trainable params: 3,407,872 || all params: 7,245,139,968 || trainable%: 0.0470


In [9]:
!wget https://raw.githubusercontent.com/tatsu-lab/stanford_alpaca/main/alpaca_data.json


--2025-05-15 08:57:37--  https://raw.githubusercontent.com/tatsu-lab/stanford_alpaca/main/alpaca_data.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22773992 (22M) [text/plain]
Saving to: ‘alpaca_data.json’

alpaca_data.json    100%[===================>]  21.72M  --.-KB/s    in 0.1s    

2025-05-15 08:57:38 (145 MB/s) - ‘alpaca_data.json’ saved [22773992/22773992]



In [10]:
from datasets import Dataset
import json

with open("alpaca_data.json", "r") as f:
    data = json.load(f)

dataset = Dataset.from_list(data)


In [11]:
def format_instruction(example):
    if example["input"]:
        prompt = f"### Instruction:\n{example['instruction']}\n\n### Input:\n{example['input']}\n\n### Response:\n"
    else:
        prompt = f"### Instruction:\n{example['instruction']}\n\n### Response:\n"
    return {"text": prompt + example["output"]}

dataset = dataset.map(format_instruction)


Map:   0%|          | 0/52002 [00:00<?, ? examples/s]

In [12]:
def tokenize(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=512)

tokenized_dataset = dataset.map(tokenize)


Map:   0%|          | 0/52002 [00:00<?, ? examples/s]

## We have:

✅ Downloaded and loaded Alpaca dataset

✅ Formatted it as instruction → response

✅ Tokenized it for your model

## Set Up the Data Collator
The model will now learn by predicting the next word, not guessing [MASK] tokens.

In [13]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Causal language modeling = predict next token
)


## Define TrainingArguments

In [14]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./mistral-qlora-instruct",
    per_device_train_batch_size=2,            # If you're on Colab T4
    gradient_accumulation_steps=4,            # Simulates larger batch
    learning_rate=2e-4,
    num_train_epochs=1,                       # Start with 1 for testing
    fp16=True,                                # Use GPU precision
    logging_steps=10,
    save_steps=50,
    save_total_limit=2,
    report_to="none"                          # No WANDB, simple logs
)


## Set Up the Trainer
This combines the model, dataset, and training settings into one object that manages training.

In [15]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)


<ipython-input-15-8d97f0df14f2>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


## Start Fine-Tuning

In [ ]:
trainer.train()


Step,Training Loss
10,1.473700
20,1.306900
30,1.239100
40,1.232800
50,1.052600
60,1.048400
70,1.183900
80,1.121800
90,1.175200
100,1.215000


## Save LoRA-Tuned Model
saves only the LoRA adapter weights

In [ ]:
model.save_pretrained("./mistral-lora-instruct")
tokenizer.save_pretrained("./mistral-lora-instruct")


## Reuse It Later
load and use your instruction-tuned model -

In [ ]:
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

base_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-v0.1",
    quantization_config=bnb_config,
    device_map="auto"
)

model = PeftModel.from_pretrained(base_model, "./mistral-lora-instruct")
tokenizer = AutoTokenizer.from_pretrained("./mistral-lora-instruct")


In [ ]:
inputs = tokenizer("### Instruction:\nTell me a joke.\n\n### Response:\n", return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


✅ What we did —

🔹 Step 1: Loaded a Big Pretrained Model (Mistral 7B)
You loaded Mistral with:

4-bit quantization (very memory-efficient)

On free Colab GPU
🔧 This gives a super-smart model but very lightweight!

🔹 Step 2: Added LoRA Adapters
Instead of changing the huge model, added small "notes" (LoRA adapters) to:

Make tiny changes

Only train a few new weights
📌 Like writing sticky notes on a textbook instead of rewriting the whole book.

🔹 Step 3: Loaded the Alpaca Dataset
You downloaded instruction examples like:

json
Copy
Edit
{
  "instruction": "Describe a cat.",
  "input": "",
  "output": "A cat is a small furry animal often kept as a pet."
}
📌 These teach the model how to follow commands.

🔹 Step 4: Formatted the Data for Instruction Tuning
You turned each row into a prompt like:

text
Copy
Edit
### Instruction:
Describe a cat.

### Response:
A cat is a small furry animal...
📌 This teaches the model to reply like a chatbot.


🔹 Step 5: Tokenized the Text
You converted the text into numbers (tokens) the model understands.

📌 This is like translating human words into computer language.


🔹 Step 6: Prepared the Training Settings
You told the model:

Use small batches

Use 1–3 training loops (epochs)

Print progress

Save the results

📌 This is like setting the rules for a classroom session.

🔹 Step 7: Started Fine-Tuning!
You used:

python
Copy
Edit
trainer.train()
This:

Showed your model hundreds of examples

Let it learn to follow instructions

Stored the new LoRA adapter (tiny weight updates)

🎉 Your model learned to follow instructions like ChatGPT, using your data.

🔹 Step 8: Saved the Fine-Tuned Model
You ran:

python
Copy
Edit
model.save_pretrained("mistral-lora-instruct")

📌 Now we can reuse your smart assistant later!